# WAVES Summer School 2025 (Gandia)

This Jupyter Notebook has been designed to be run in [Google Colab](https://colab.research.google.com/). With this purpose the first cell install [NGSolve](https://ngsolve.org/) related packages in a clean machine (if they have not been previously installed). Typically, this installation takes less than 2 minutes.

In [ ]:
# Install NGSolve
try:
    import ngsolve
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/ngsolve-install-release-complex.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"

# Install NGSolve special functions
try:
    import ngsolve_special_functions
except ImportError:
    !git clone "https://github.com/NGSolve/ngs-special-functions.git" && cd ngs-special-functions && mkdir build && cd build && cmake .. && make -j4 && cp ngsolve_special_functions.so /content/radiation_test

# Linear Acoustics in the time-harmonic setting: Displacement formulation in $H(\mathrm{div})$ using Raviart-Thomas elements

This notebook illustrates the numerical solution of the wave equation for harmonic excitation using the so called [Finite Element Method](https://jschoeberl.github.io/iFEM/intro.html) (FEM). The method aims at an approximate solution by subdividing the area of interest into smaller parts with simpler geometry, linking these parts together and applying methods from the calculus of variations to solve the problem numerically. The FEM is a well established method for the numerical approximation of the solution of partial differential equations (PDEs). The solutions of PDEs are often known analytically only for rather simple geometries. FEM based simulations allow to gain insights into other more complex cases.

## Variational Formulation

The FEM is based on expressing the partial differential equation (PDE) to be solved in its variational or weak form.


## Numerical Solution

The numerical solution of the variational problem is based on [NGSolve](https://ngsolve.org/), an open-source framework for numerical solution of PDEs.
Its high-level Python interface is used in the following to define the problem and compute its solution.
The implementation is based on the variational formulation derived above. The definition of the problem in NGsolve is very close to the mathematical formulation of the problem. A function `FEM_displacements(...)` is defined to analyze the convergence beahviour and the optimal strategy: $h$ or $p$-refinement.

In [ ]:
# Libraries 
import time
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Geometry
R_int = 1.0
R_ext = 2.0

# Primary geometric objects
square= Rectangle(R_ext, R_ext).Face()
circ_int = Circle((0,0), R_int).Face()
circ_ext = Circle((0,0), R_ext).Face()

# Domain: A quarter of an annular domain
domain = square * (circ_ext - circ_int) 

# Boundary tags
tol=1e-1
domain.faces.name = "fluid"
domain.edges.name = "exterior"
domain.edges.col = (1, 0, 0) # red
domain.edges.Min(Y).name = "horizontal"
domain.edges.Min(Y).col = (1, 0, 1) # magenta
domain.edges.Min(X).name = "vertical"
domain.edges.Min(X).col = (0, 1, 1) # cyan
domain.edges[sqrt(X*X+Y*Y)<R_int+tol].name = "interior"
domain.edges[sqrt(X*X+Y*Y)<R_int+tol].col = (1, 1, 0) # yellow
domain.mat("fluid")

## Each edge is colored
Draw(domain, height="3vh")

In [ ]:

# Define the mesh
h_size = 0.3
mesh = Mesh(OCCGeometry(domain, dim=2).GenerateMesh(maxh=h_size))
mesh.Curve(3)  # Refine the mesh to cubic elements
Draw(mesh, height="3vh")

Exact solution and boundary data for getting a solution in closed form

In [ ]:
from ngsolve_special_functions import hankel1

# Physical parameters
f = 100 #500 # Frequency in Hz
rho = 1.0 # Density in kg/m^3
c = 343.0 # Speed of sound in m/s

# Pressure on the interior boundary
omega = 2*np.pi*f
pex = hankel1(z=omega/c*sqrt(x**2+y**2), v=0) # pressure
e_r = CF((x,y))/sqrt(x**2+y**2) # radial unit vector
uex = 1.0/rho/omega**2*(-omega/c)*hankel1(z=omega/c*sqrt(x**2+y**2), v=1)*e_r # displacement
div_uex = -pex/rho/c**2 # divergence of the displacement

# Plot the exact solution
Draw(uex[0], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})
Draw(uex[1], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})

# Boundary data for the pressure in the interior boundary
p_bnd = pex  # CF(pex(mesh(R_int, 0.)))

# Boundary data for the displacement in the interior boundary
u_bnd = uex  # CF(uex(mesh(R_int, 0.)))

# Surface impedance data on the exterior boundary
Z_bnd = CF(pex(mesh(R_ext, 0.))/(-1j*omega*uex(mesh(R_ext, 0.))[0])) # pex/(-1j*omega*uex*e_r) 


### Finite element computation using Raviart-Thomas elements

In [ ]:
# Compute the Finite Element approximation
def FEM_displacements(mesh, order_FE, f, rho, c, p_bnd, u_bnd, Z_bnd):

    # Angular frequency
    omega= 2*np.pi*f

    # Raviart-Thomas finite element with complex values to approximate the displacement field
    V = HDiv(mesh, order=order_FE, RT = True, complex=True, dirichlet = "horizontal|vertical")

    # Trial and test functions
    u, v = V.TnT()

    # Normal vector outward to the boundary
    normal = specialcf.normal(mesh.dim)

    # Define the differential on the integration on the volumes
    dF = dx(mesh.Materials("fluid")) # fluid domain

    # Define the differential on the integration on the boundary
    dI = ds(mesh.Boundaries("interior")) # interior boundary
    dE = ds(mesh.Boundaries("exterior")) # exterior boundary

    # Variational formulation: Bilinear form
    a_bilinear = BilinearForm(V, symmetric=False)

    # Contribution in the air and the impedance boundary
    a_bilinear += rho*c**2*div(u)*div(v)*dF - omega**2*rho*u*v*dF -1j*omega*Z_bnd*u.Trace()*v.Trace()*dE

    # Forma lineal
    f_linear = LinearForm(V)

    # Contribution of the pressure on the interior part
    f_linear += p_bnd*(v.Trace()*normal)*dI

    # Assembly the FEM matrices and the right-hand side
    a_bilinear.Assemble()
    f_linear.Assemble()

    # Allocate the solution vector with the prescribed Dirichlet boundary conditions
    gfu = GridFunction(V)
    gfu.Set(u_bnd, definedon=mesh.Boundaries("horizontal|vertical"))

    # Solve the linear system
    precond = Preconditioner(a_bilinear,"direct") # sparse direct solver (UMFPACK)
    solvers.BVP(bf=a_bilinear, lf=f_linear, gf=gfu, pre=precond, print=False)

    return gfu, V

Function to compute the RMS errors and the errors associated with the acoustic energy of the system

In [ ]:
# Compute the root-mean square (RMS) relative error
def compute_error(rho, c, gfu, uex, div_uex, V):

    # RMS of the exact solution (pressure)
    RMS_p_ex = sqrt(Integrate(rho*omega**2*InnerProduct(uex, uex), V.mesh, order=2*V.globalorder))

    # RMS absolute error between the exact and the Finite Element approximation (pressure)
    error_RMS_p = sqrt(Integrate(rho*omega**2*InnerProduct(gfu-uex, gfu-uex), V.mesh, order=2*V.globalorder))

    # Energy of the exact solution (velocity)
    RMS_v_ex = sqrt(Integrate(rho*c**2*InnerProduct(div_uex, div_uex), V.mesh, order=2*V.globalorder))

    # Energy of the absolute error between the exact and the Finite Element approximation
    error_RMS_v = sqrt(Integrate(rho*c**2*InnerProduct(div(gfu)-div_uex, div(gfu)-div_uex), V.mesh, order=2*V.globalorder))
    
    # Energy associated to the exact solution
    energy_ex = sqrt(RMS_p_ex**2 + RMS_v_ex**2)

    # Energy error
    error_energy = sqrt(error_RMS_p**2 + error_RMS_v**2)

    return error_RMS_p.real/RMS_p_ex.real, error_RMS_v.real/RMS_v_ex.real, error_energy.real/energy_ex.real

### Study of convergence

Let's check the order of convergence of this finite element discretization computing the error for sucesive refinements of the mesh

In [ ]:
# Finite element order
order_FE = np.arange(1, 5)  # Orders from 1 to 4
h_size = np.array([0.4, 0.2, 0.1, 0.05])  # Mesh sizes

# Compute the Finite Element approximation and the error for different mesh sizes and orders
error_RMS_p = np.zeros((len(h_size), len(order_FE)))
error_RMS_v = np.zeros((len(h_size), len(order_FE)))
error_energy = np.zeros((len(h_size), len(order_FE)))
wall_time = np.zeros((len(h_size), len(order_FE)))

# Loop over mesh sizes and finite element orders
for i, hs in enumerate(h_size):
    mesh = Mesh(OCCGeometry(domain, dim=2).GenerateMesh(maxh=hs))
    for j, order in enumerate(order_FE):
        mesh.Curve(int(order))
        start_wall = time.time()
        gfu, V = FEM_displacements(mesh, int(order), f, rho, c, p_bnd, u_bnd, Z_bnd)
        wall_time[i, j] = time.time() - start_wall
        error_RMS_p[i, j], error_RMS_v[i, j], error_energy[i, j] = compute_error(rho, c, gfu, uex, div_uex, V)
        # Print the errors
        print(f"Order {order}, Mesh size {hs}: RMS Error (Pressure) = {error_RMS_p[i, j]:.4e}, RMS Error (Velocity) = {error_RMS_v[i, j]:.4e}, Energy Error = {error_energy[i, j]:.4e}")

# Plot last pressure field computed
Draw(gfu[0], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})
Draw(gfu[1], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})

Compute the order of convergence of the FEM discretizations

In [ ]:
# Get order of convergence for each order_FE
for j, order in enumerate(order_FE):
    mRMS_p, _ = np.polyfit(np.log(h_size[1:]), np.log(error_RMS_p[1:, j]), 1)
    mRMS_v, _ = np.polyfit(np.log(h_size[1:]), np.log(error_RMS_v[1:, j]), 1)
    print(f"Order {order}: RMS Error convergence rate (pressure) = {mRMS_p:.1f}, RMS Error convergence rate (velocity) = {mRMS_v:.1f}")
    mEnergy, _ = np.polyfit(np.log(h_size[1:]), np.log(error_energy[1:, j]), 1)
    print(f"Order {order}: Energy Error convergence rate = {mEnergy:.1f}")

Check the numerical errors and observe the FEM convergence

In [ ]:
# Plot the convergence results with tag the order in the legend
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.loglog(h_size*f/c, 100*error_RMS_p, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('RMS Rel. Error (%)')
plt.title('RMS Rel. Error (Pressure) vs h/λ')
plt.grid(True)
plt.legend()
plt.subplot(1, 3, 2)
plt.loglog(h_size*f/c, 100*error_RMS_v, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('RMS Rel. Error (%)')
plt.title('RMS Rel. Error (Velocity) vs h/λ')
plt.grid(True)
plt.legend()
plt.subplot(1, 3, 3)
plt.loglog(h_size*f/c, 100*error_energy, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('Energy Rel. Error (%)')
plt.title('Energy Rel. Error vs h/λ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Checking the optimal strategy: refine the mesh or increasing the FE order?

In [ ]:
# Plot the wall time vs energy relative error with tag the order in the legend
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.loglog(100*error_energy, wall_time, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('Energy Relative Error (%)')
plt.ylabel('Wall Time (s)')
plt.title('Wall Time vs Energy Relative Error')
plt.grid(True)
plt.legend()
plt.subplot(1, 2, 2)
plt.loglog(100*error_energy.T, wall_time.T, marker='o', label=[f"h={hs}" for hs in h_size])
plt.xlabel('Energy Relative Error (%)')
plt.ylabel('Wall Time (s)')
plt.title('Wall Time vs Energy Relative Error')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Write to a csv file the wall time and the RMS and energy errors

In [ ]:
# Write to a CSV file the errors computed with the RMS and energy norms
import pandas as pd
# Create a DataFrame with the errors and wall time
df_errors = pd.DataFrame({
    'Mesh Size (h)': np.tile(h_size, len(order_FE)),
    'Order': np.repeat(order_FE, len(h_size)),
    'RMS Error pressure': error_RMS_p.flatten(),
    'RMS Error velocity': error_RMS_v.flatten(),
    'Energy Error': error_energy.flatten(),
    'Wall Time (s)': wall_time.flatten()
})
# Save the DataFrame to a CSV file
df_errors.to_csv('HDiv-velocity.csv', index=False)


**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).